01) Design model (input, output size, forward pass)

02) Construct loss and optimizer

03) Training loop
    - forward pass: compute prediction
    - backward pass: gradients
    - update weights

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

C:\Users\Vidush\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [6]:
n_samples , n_features = X.shape
print(n_samples, n_features)

569 30


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [8]:
# scale
sc = StandardScaler()

In [9]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [11]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [12]:
class LogosticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogosticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [13]:
model = LogosticRegression(n_features)

In [14]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
num_epochs = 100



for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    
    # backward pass
    loss.backward()
    
    # updates
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 0.4339
epoch: 20, loss = 0.3840
epoch: 30, loss = 0.3477
epoch: 40, loss = 0.3199
epoch: 50, loss = 0.2978
epoch: 60, loss = 0.2796
epoch: 70, loss = 0.2644
epoch: 80, loss = 0.2515
epoch: 90, loss = 0.2402
epoch: 100, loss = 0.2304
accuracy = 0.8772
